In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import sympy as sp
from tensorflow.keras.models import load_model

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

2024-07-01 17:17:46.788032: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-01 17:17:46.812159: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-01 17:17:46.851108: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-01 17:17:46.851158: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-01 17:17:46.871210: I tensorflow/core/platform/cpu_feature_gua

In [2]:
class_names = ['+', '-', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '=', 'times']

In [9]:
model = load_model('../../models/eqn-detect-model.keras')

2024-07-01 17:18:19.907036: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2024-07-01 17:18:19.907057: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:134] retrieving CUDA diagnostic information for host: pablo-sene
2024-07-01 17:18:19.907060: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:141] hostname: pablo-sene
2024-07-01 17:18:19.907176: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:165] libcuda reported version is: 535.183.1
2024-07-01 17:18:19.907189: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:169] kernel reported version is: 535.183.1
2024-07-01 17:18:19.907191: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:248] kernel version seems to match DSO: 535.183.1


In [10]:
def binarize(img):
    # Convert to grayscale
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Perform adaptive thresholding
    binarized = cv2.adaptiveThreshold(gray_img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)
    
    # Invert binary image
    inverted_binary_img = cv2.bitwise_not(binarized)
    
    # Expand dimensions to make it suitable for model input
    expanded_img = np.expand_dims(inverted_binary_img, -1)
    
    return expanded_img

In [11]:
def getOverlap(x_i, x_j):
    if max(x_i) > min(x_j) and max(x_j) > min(x_i):  # Check for overlap
        return min(max(x_i) - min(x_j), max(x_j) - min(x_i))  # Return overlap size
    return 0  # No overlap

def detect_contours(frame, min_width=30, min_height=30, canny_threshold1=100, canny_threshold2=150):
    # Converter para escala de cinza
    input_image_cpy = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Aplicar o detector de bordas de Canny
    edges = cv2.Canny(input_image_cpy, canny_threshold1, canny_threshold2)
    
    # Encontrar contornos
    contours_list, hierarchy = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    bounding_boxes = []
    for i, c in enumerate(contours_list):
        x, y, w, h = cv2.boundingRect(c)
        
        # Filtrar contornos com base no tamanho mínimo
        if w >= min_width and h >= min_height:
            bounding_boxes.append([x, y, w, h, i])  # Adicionar índice do contorno
    
    filtered_boxes = []
    for box in bounding_boxes:
        x, y, w, h, idx = box
        is_within_another = False
        
        # Verificar se este contorno está dentro de outro contorno maior
        current_hierarchy = hierarchy[0][idx]
        parent_idx = current_hierarchy[3]
        
        while parent_idx != -1:
            px, py, pw, ph = cv2.boundingRect(contours_list[parent_idx])
            if pw >= min_width and ph >= min_height:
                is_within_another = True
                break
            parent_idx = hierarchy[0][parent_idx][3]
        
        if not is_within_another:
            filtered_boxes.append([x, y, w, h])
    
    return filtered_boxes


In [12]:
def resize_pad(img, size, padColor=255):
    h, w = img.shape[:2]
    sh, sw = size

    # interpolation method
    if h > sh or w > sw: # shrinking image
        interp = cv2.INTER_AREA
    else: # stretching image
        interp = cv2.INTER_CUBIC

    # aspect ratio of image
    aspect = w / h

    # compute scaling and pad sizing
    if aspect > 1: # horizontal image
        new_w = sw
        new_h = int(round(new_w / aspect))
        pad_vert = (sh - new_h) / 2
        pad_top, pad_bot = int(np.floor(pad_vert)), int(np.ceil(pad_vert))
        pad_left, pad_right = 0, 0
    elif aspect < 1: # vertical image
        new_h = sh
        new_w = int(round(new_h * aspect))
        pad_horz = (sw - new_w) / 2
        pad_left, pad_right = int(np.floor(pad_horz)), int(np.ceil(pad_horz))
        pad_top, pad_bot = 0, 0
    else: # square image
        new_h, new_w = sh, sw
        pad_left, pad_right, pad_top, pad_bot = 0, 0, 0, 0

    # set pad color
    if len(img.shape) == 3 and not isinstance(padColor, (list, tuple, np.ndarray)):
        padColor = [padColor] * 3

    # scale and pad
    scaled_img = cv2.resize(img, (new_w, new_h), interpolation=interp)
    scaled_img = cv2.copyMakeBorder(scaled_img, pad_top, pad_bot, pad_left, pad_right, borderType=cv2.BORDER_CONSTANT, value=padColor)

    return scaled_img



In [13]:
# get the IMAGE from webcam in real time, show the equation in the webcam in real time
eqn_list = []
def preprocess_webcam_image(img):
    inverted_binary_img = binarize(img)
    keep = detect_contours(img)
    for (x, y, w, h) in sorted(keep, key = lambda x: x[0]):
        img = resize_pad(inverted_binary_img[y:y+h, x:x+w], (45, 45), 0) # We must use the binarized image to predict
        pred_class = class_names[np.argmax(model.predict(tf.expand_dims(tf.expand_dims(img, 0), -1)))]
        
        if pred_class == "times":
            pred_class = "*"
        eqn_list.append(pred_class)
        # print(eqn_list)
    eqn = "".join(eqn_list)
    
    return eqn

In [14]:
def solve_equation(equation):
    # Trata casos de existência de igualdade sem valor
    if "=" in equation:
        # Dividir a equação em lado esquerdo e direito
        left, right = equation.split("=")
        left = left.strip()
        right = right.strip()
        if not left and not right:
            raise("Não tem equação")
        if not left:
            equation = right
        elif not right:
            equation = left
        else:
            equation = equation

    # Detectar o caso baseado no conteúdo da equação
    if isinstance(equation, list):
        left_expr = equation[0].lhs
        right_expr = equation[0].rhs
    else:
        if "=" in str(equation):
            left_expr = equation.lhs
            right_expr = equation.rhs
        else:
            left_expr = equation
            right_expr = sp.S.Zero

    # Tentar converter as expressões para símbolos
    try:
        left_expr = sp.sympify(left_expr)
        right_expr = sp.sympify(right_expr)
    except sp.SympifyError:
        return "Equação inválida"

    # Caso 1: equação apenas numérica
    if left_expr.is_number and right_expr.is_number:
        result = left_expr - right_expr
        return result.evalf()  # Usar evalf() para obter o valor numérico

    # Caso 2 e 3: equações com variáveis
    variables = list(left_expr.free_symbols | right_expr.free_symbols)

    # Resolver a equação usando solve
    solutions = sp.solve(left_expr - right_expr, variables)

    # Melhorar a legibilidade das soluções
    numeric_solutions = [sol.evalf() for sol in solutions]
    
    return numeric_solutions


In [15]:
def most_frequent(List):
    return max(set(List), key = List.count)

cap = cv2.VideoCapture(2)

if not cap.isOpened():
    print("Erro ao abrir a câmera.")
else:
    count = 0 
    eqn_list = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        

        eqn = preprocess_webcam_image(frame)
        eqn_list.append(eqn)
        # Process frame to detect contours
        detected_boxes = detect_contours(frame)

        for box in detected_boxes:
            x, y, w, h = box
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Green rectangle


        if count % 20 == 0:
            print(eqn_list)
            real_eqn = most_frequent(eqn_list)
            print(real_eqn)
            if real_eqn:
                result = solve_equation(real_eqn)
            else:
                result = "Nao foi encontrada uma equacao"
            eqn_list = []
        
        # Display the frame with detected contours
        if result == "Nao foi encontrada uma equacao":
            cv2.putText(frame, f"{result}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        else:
            cv2.putText(frame, f"{real_eqn} = {result:.5}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        count += 1
        cv2.imshow('Detected Contours', frame)
    

        # Exit the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

Erro ao abrir a câmera.


[ WARN:0@38.067] global cap_v4l.cpp:997 open VIDEOIO(V4L2:/dev/video2): can't open camera by index
[ERROR:0@38.183] global obsensor_uvc_stream_channel.cpp:159 getStreamChannelGroup Camera index out of range
